In [4]:
import os
import sys
import pandas as pd
import numpy as np
from ezchord import Chord
from mingus.core.notes import int_to_note

sys.path.append('..')
from melody import Melody
from utils import get_chord_progressions, is_weakly_polyphonic, is_strongly_polyphonic

In [5]:
bpm = 4

cp = get_chord_progressions('..')['A Felicidade']

linear_chord_progression = []

for section in cp['sections']:
    linear_chord_progression += cp['progression'][section]

n_chord_prog_measures = int(len(linear_chord_progression) / bpm)

In [6]:
orig = pd.read_csv('../data/split_melody_data/v1.2/Real Book/A Felicidade -o-.csv', index_col=0)
orig['end_ticks'] = (orig['ticks'] + orig['duration'])
orig.sort_values('ticks', inplace=True)

In [7]:
orig.shape

(126, 13)

In [8]:
impro = pd.read_csv('../data/split_melody_data/v1.2/JazzPage/A Felicidade -1-.csv', index_col=0)
impro['end_ticks'] = (impro['ticks'] + impro['duration'])
impro.sort_values('ticks', inplace=True)

In [9]:
n_chord_prog_measures

48

In [10]:
def multiple_pitches_to_string(pitches):
    return "-".join(str(x) for x in pitches)

In [11]:
def extract_poly_time_step_encoding(impro, orig, linear_chord_progression):
    tpm = 48
    n_chord_prog_beats = len(linear_chord_progression)
    n_ticks = tpm * n_chord_prog_measures

    rows = []

    for i in range(n_ticks):
        offset = i % tpm

        impro_sustains = impro[(impro['ticks'] < i) & (i < impro['end_ticks'])]['pitch'].values
        impro_sustains = multiple_pitches_to_string(impro_sustains)

        impro_attacks = impro[impro['ticks'] == i]['pitch'].values
        impro_attacks = multiple_pitches_to_string(impro_attacks)

        orig_sustains = orig[(orig['ticks'] < i) & (i < orig['end_ticks'])]['pitch'].values
        orig_sustains = multiple_pitches_to_string(orig_sustains)

        orig_attacks = orig[orig['ticks'] == i]['pitch'].values
        orig_attacks = multiple_pitches_to_string(orig_attacks)

        chord_name = linear_chord_progression[(i // (48 // bpm))]   

        rows.append({
            'offset': offset,

            'impro_sustains': impro_sustains,
            'impro_attacks': impro_attacks,

            'orig_sustains': orig_sustains,
            'orig_attacks': orig_attacks,

            'chord_name': chord_name
        })
        
    return pd.DataFrame(rows)

In [12]:
def remove_strong_polyphony(melody):
    mono_melody = melody.copy()
    
    mono_melody.groupby('ticks').apply(lambda : x['pitch'].max())
    
    return mono_melody

In [13]:
def remove_weak_polyphony(melody):
    mono_melody = melody.copy()

    overlap = (mono_melody['end_ticks'] - mono_melody['ticks'].shift(-1)).clip(0, None)
    

    # skip last row as 'shift' messes it up
    mono_melody['duration'].iloc[:-1] -= overlap.iloc[:-1]
    mono_melody['end_ticks'].iloc[:-1] -= overlap.iloc[:-1]
    

    return mono_melody

In [14]:
def extract_mono_time_step_encoding(impro, orig, linear_chord_progression):
    tpm = 48
    n_chord_prog_beats = len(linear_chord_progression)
    n_ticks = tpm * n_chord_prog_measures

    rows = []

    if is_strongly_polyphonic(impro):
        print('Impro SP')
        impro = remove_strong_polyphony(impro)
        
        if is_strongly_polyphonic(impro):
            raise Exception('Error!!! P')

    if is_weakly_polyphonic(impro):
        print('Impro WP')
        impro = remove_weak_polyphony(impro)
               

        if is_weakly_polyphonic(impro):
            raise Exception('Error!!! W')

    if is_strongly_polyphonic(orig):
        print('Original SP')
        
        orig = remove_strong_polyphony(orig)
        
        if is_strongly_polyphonic(impro):
            raise Exception('Error!!! P')
            
    if is_weakly_polyphonic(orig):
        print('Original WP')
        
        orig = remove_weak_polyphony(orig)

        if is_weakly_polyphonic(orig):
            raise Exception('Error!!! W')

    for i in range(n_ticks):
        offset = i % tpm

        impro_pitch = np.nan
        impro_pitches = impro[(impro['ticks'] <= i) & (i < impro['end_ticks'])]

        if len(impro_pitches) > 0:
            impro_pitch = impro_pitches['pitch'].values[0]

            if len(impro_pitches) > 1:
                raise Exception('Error!!! not mono pitch')
                
        impro_attack = 0
        impro_attacks = impro[impro['ticks'] == i]

        if len(impro_attacks) > 0:
            impro_attack = 1

            if len(impro_attacks) > 1:
                raise Exception('Error!!! not mono attack')

        orig_pitch = np.nan
        orig_pitches = orig[(orig['ticks'] <= i) & (i < orig['end_ticks'])]

        if len(orig_pitches) > 0:
            orig_pitch = orig_pitches['pitch'].values[0]

            if len(orig_pitches) > 1:
                print(orig_pitches[['pitch', 'ticks', 'end_ticks']])
                raise Exception('Error!!! not mono pitch')

        orig_attack = 0
        orig_attacks = orig[orig['ticks'] == i]

        if len(orig_attacks) > 0:
            orig_attack = 1

            if len(orig_attacks) > 1:
                raise Exception('Error!!! not mono attack')
  
        chord_name = linear_chord_progression[(i // (48 // bpm))]   
        
        rows.append({
            'offset': offset,

            'impro_pitch': impro_pitch,
            'impro_attack': impro_attack,

            'orig_pitch': orig_pitch,
            'orig_attack': orig_attack,

            'chord_name': chord_name
        })

    return pd.DataFrame(rows)

In [15]:
mono = extract_mono_time_step_encoding(impro, orig, linear_chord_progression)

Impro WP
Original WP


/tmp/ipykernel_11409/3749693121.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mono_melody['duration'].iloc[:-1] -= overlap.iloc[:-1]
/tmp/ipykernel_11409/3749693121.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mono_melody['end_ticks'].iloc[:-1] -= overlap.iloc[:-1]
/tmp/ipykernel_11409/3749693121.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mono_melody['duration'].iloc[:-1] -= overlap.iloc[:-1]
/tmp/ipykernel_11409/

In [19]:
poly = extract_poly_time_step_encoding(impro, orig, linear_chord_progression)

In [20]:
p = set(poly[poly['orig_attacks']!=''].index)

In [21]:
m = set(mono[mono['orig_attack']!=0].index)

In [22]:
pd.concat([mono, poly],axis=1).loc[list(p.difference(m))][['orig_attacks', 'orig_attack']]

,orig_attacks,orig_attack


In [23]:
import os

mono_filepath = '../data/encoded/timestep/mono/JazzPage/'
if not os.path.exists(mono_filepath):
    os.makedirs(mono_filepath)
    
poly_filepath = '../data/encoded/timestep/poly/JazzPage/'
if not os.path.exists(poly_filepath):
    os.makedirs(poly_filepath)

mono.to_csv(mono_filepath + 'A Felicidade -1-.csv')
poly.to_csv(poly_filepath + 'A Felicidade -1-.csv')

In [16]:
mono[mono['impro_attack'] == True].tail(5)

,offset,impro_pitch,impro_attack,orig_pitch,orig_attack,chord_name
2250,42,80.0,1,57.0,0,Amin7
2256,0,81.0,1,57.0,0,Amin7
2268,12,83.0,1,57.0,0,Amin7
2280,24,84.0,1,57.0,0,Amin7
2292,36,86.0,1,64.0,1,Amin7


In [ ]:
impro.tail(5)

In [ ]:
len(linear_chord_progression)

In [ ]:
mono.tail()

In [17]:
mono

,offset,impro_pitch,impro_attack,orig_pitch,orig_attack,chord_name
0,0,NaN,0,69.0,1,Amin7
1,1,NaN,0,69.0,0,Amin7
2,2,NaN,0,69.0,0,Amin7
3,3,NaN,0,69.0,0,Amin7
4,4,NaN,0,69.0,0,Amin7
...,...,...,...,...,...,...
2299,43,86.0,0,64.0,0,Amin7
2300,44,86.0,0,64.0,0,Amin7
2301,45,86.0,0,64.0,0,Amin7
2302,46,NaN,0,64.0,0,Amin7


In [50]:
mono[np.isnan(mono['impro_pitch'])]

,offset,impro_pitch,impro_attack,orig_pitch,orig_attack,chord_name,idx
0,0,NaN,0,69.0,1,Amin7,0
1,1,NaN,0,69.0,0,Amin7,1
2,2,NaN,0,69.0,0,Amin7,2
3,3,NaN,0,69.0,0,Amin7,3
4,4,NaN,0,69.0,0,Amin7,4
...,...,...,...,...,...,...,...
2277,21,NaN,0,57.0,0,Amin7,2277
2278,22,NaN,0,57.0,0,Amin7,2278
2279,23,NaN,0,57.0,0,Amin7,2279
2302,46,NaN,0,64.0,0,Amin7,2302


In [21]:
mono['idx'] = mono.index

In [37]:
aaa = mono[np.isnan(mono['impro_pitch'])]

In [42]:
aaa = aaa.reset_index(drop=True)

In [48]:
(aaa['idx'] - aaa.index).head(50)

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
30     0
31     0
32     0
33     0
34     0
35     0
36     0
37     0
38     0
39     0
40     0
41     0
42     0
43     4
44     4
45     4
46     4
47    19
48    24
49    24
dtype: int64

In [54]:
m = np.isnan(mono['impro_pitch'])

mono['rep'] = m.ne(m.shift() & m).cumsum()

In [55]:
mono

,offset,impro_pitch,impro_attack,orig_pitch,orig_attack,chord_name,idx,rep
0,0,NaN,0,69.0,1,Amin7,0,1
1,1,NaN,0,69.0,0,Amin7,1,1
2,2,NaN,0,69.0,0,Amin7,2,1
3,3,NaN,0,69.0,0,Amin7,3,1
4,4,NaN,0,69.0,0,Amin7,4,1
...,...,...,...,...,...,...,...,...
2299,43,86.0,0,64.0,0,Amin7,2299,164
2300,44,86.0,0,64.0,0,Amin7,2300,164
2301,45,86.0,0,64.0,0,Amin7,2301,164
2302,46,NaN,0,64.0,0,Amin7,2302,165


In [58]:
mono.head(50)

,offset,impro_pitch,impro_attack,orig_pitch,orig_attack,chord_name,idx,rep
0,0,NaN,0,69.0,1,Amin7,0,1
1,1,NaN,0,69.0,0,Amin7,1,1
2,2,NaN,0,69.0,0,Amin7,2,1
3,3,NaN,0,69.0,0,Amin7,3,1
4,4,NaN,0,69.0,0,Amin7,4,1
5,5,NaN,0,69.0,0,Amin7,5,1
6,6,NaN,0,69.0,0,Amin7,6,1
7,7,NaN,0,69.0,0,Amin7,7,1
8,8,NaN,0,69.0,0,Amin7,8,1
9,9,NaN,0,69.0,0,Amin7,9,1
